In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
%load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
query2=f"""
SELECT
A.Year,
A.GP,
A.Driver,
A.DriverNumber,
A.Team,
CAST(A.Position AS INT64) Position,
Status,
A.LapStartTime,
A.Time LapEndTime,
B.Time as LapInstant,
A.LapTime,
CAST(LapNumber as INT64) LapNumber,
RPM,
Speed,
nGear,
Throttle,
DRS,
Distance,
RelativeDistance,
DriverAhead,
DistanceToDriverAhead,
X,
Y,
Z,
CAST(Stint as INT64) Stint,
PitOutTime,
PitInTime,
Sector1Time,
Sector2Time,
Sector3Time,
Sector1SessionTime,
Sector2SessionTime,
Sector3SessionTime,
CAST(SpeedI1 AS INT64) SpeedI1,
CAST(SpeedI2 AS INT64) SpeedI2,
CAST(SpeedFL AS INT64) SpeedFL,
CAST(SpeedST AS INT64) SpeedST,
Compound,
CAST(TyreLife AS INT64) TyreLife,
FreshTyre,
TrackStatus,
Deleted,
DeletedReason
FROM `perceptive-ivy-290216.f1_api.race_lap_time`  A
INNER JOIN
`perceptive-ivy-290216.f1_api.f1_race_telemetry_2021` B
ON B.SessionTime>=A.LapStartTime AND B.SessionTime<=A.Time
AND A.Driver=B.Driver
AND A.Year=B.Year
AND A.GP=B.GP
WHERE A.Year=2021
AND A.GP="Abu Dhabi Grand Prix"
AND A.DRIVER IN ('SAI')
AND B.Year=2021
AND B.GP="Abu Dhabi Grand Prix"
AND B.Driver IN ('SAI')
ORDER BY LapNumber, LapStartTime, LapInstant
"""
track_1=pd.read_gbq(query2,project_id,dialect='standard')

In [ ]:
query3=f"""
SELECT
A.Year,
A.GP,
A.Driver,
A.DriverNumber,
A.Team,
CAST(A.Position AS INT64) Position,
Status,
A.LapStartTime,
A.Time LapEndTime,
B.Time as LapInstant,
A.LapTime,
CAST(LapNumber as INT64) LapNumber,
RPM,
Speed,
nGear,
Throttle,
DRS,
Distance,
RelativeDistance,
DriverAhead,
DistanceToDriverAhead,
X,
Y,
Z,
CAST(Stint as INT64) Stint,
PitOutTime,
PitInTime,
Sector1Time,
Sector2Time,
Sector3Time,
Sector1SessionTime,
Sector2SessionTime,
Sector3SessionTime,
CAST(SpeedI1 AS INT64) SpeedI1,
CAST(SpeedI2 AS INT64) SpeedI2,
CAST(SpeedFL AS INT64) SpeedFL,
CAST(SpeedST AS INT64) SpeedST,
Compound,
CAST(TyreLife AS INT64) TyreLife,
FreshTyre,
TrackStatus,
Deleted,
DeletedReason
FROM `perceptive-ivy-290216.f1_api.race_lap_time`  A
INNER JOIN
`perceptive-ivy-290216.f1_api.f1_race_telemetry_2021` B
ON B.SessionTime>=A.LapStartTime AND B.SessionTime<=A.Time
AND A.Driver=B.Driver
AND A.Year=B.Year
AND A.GP=B.GP
WHERE A.Year=2021
AND A.GP="Abu Dhabi Grand Prix"
AND A.DRIVER IN ('VER')
AND B.Year=2021
AND B.GP="Abu Dhabi Grand Prix"
AND B.Driver IN ('VER')
ORDER BY LapNumber, LapStartTime, LapInstant
"""
track_2=pd.read_gbq(query3,project_id,dialect='standard')

In [ ]:
track_final=pd.concat([track_1, track_2])
# track2['LapInstant'].replace('0 days 00:00:00', '0 days 00:00:00.000000', inplace=True)
# track2['LapInstant'].replace('0 days 00:00:20', '0 days 00:00:00.200000', inplace=True)

In [ ]:
#USE THIS WHEN YOU WANT MORE THAN 2 DRIVERS' DATA AT ONCE. THEN COMMENT THE ABOVE 2 SQL QUERIES AND USE THIS

# query3=f"""
# SELECT
# A.Year,
# A.GP,
# A.Driver,
# B.Driver B_Driver,
# A.DriverNumber,
# A.Team,
# CAST(A.Position AS INT64) Position,
# Status,
# A.LapStartTime,
# A.Time LapEndTime,
# B.Time as LapInstant,
# A.LapTime,
# CAST(LapNumber as INT64) LapNumber,
# RPM,
# Speed,
# nGear,
# Throttle,
# DRS,
# Distance,
# RelativeDistance,
# DriverAhead,
# DistanceToDriverAhead,
# X,
# Y,
# Z,
# CAST(Stint as INT64) Stint,
# PitOutTime,
# PitInTime,
# Sector1Time,
# Sector2Time,
# Sector3Time,
# Sector1SessionTime,
# Sector2SessionTime,
# Sector3SessionTime,
# CAST(SpeedI1 AS INT64) SpeedI1,
# CAST(SpeedI2 AS INT64) SpeedI2,
# CAST(SpeedFL AS INT64) SpeedFL,
# CAST(SpeedST AS INT64) SpeedST,
# Compound,
# CAST(TyreLife AS INT64) TyreLife,
# FreshTyre,
# TrackStatus,
# Deleted,
# DeletedReason
# FROM `perceptive-ivy-290216.f1_api.race_lap_time`  A
# INNER JOIN
# `perceptive-ivy-290216.f1_api.f1_race_telemetry_2021` B
# ON B.SessionTime>=A.LapStartTime AND B.SessionTime<=A.Time
# AND A.Driver=B.Driver
# AND A.Year=B.Year
# AND A.GP=B.GP
# WHERE A.Year=2021
# AND A.GP="Abu Dhabi Grand Prix"
# -- AND A.DRIVER IN ('HAM')
# AND B.Year=2021
# AND B.GP="Abu Dhabi Grand Prix"
# -- AND B.Driver IN ('HAM')
# AND A.Driver=B.Driver
# ORDER BY Driver, LapNumber, LapStartTime, LapInstant
# """
# track_final=pd.read_gbq(query3,project_id,dialect='standard')

In [ ]:
# track2=track_final[track_final["Driver"].isin(["HAM","VER","SAI","BOT"])]
track2=track_final

In [ ]:
track2["LapTime"]=track2['LapTime'].str.split('days ').str[1]
track2["LapTime"]=pd.to_datetime(track2['LapTime'],format='%H:%M:%S.%f')
track2["LapTime"]=track2["LapTime"].dt.strftime("%H:%M:%S.%f")
track2["LapInstant"]=track2['LapInstant'].str.split('days ').str[1]

In [ ]:
#Convert lapnumber to string for discrete chart coloring
track2["LapNumber"]=track2["LapNumber"].astype(str)
# track2=track2.sort_values(by=['Driver','LapInstant'])

In [ ]:
#Calculate the distance traversed in totality and then for each lap
track2['X_Diff'] = track2['X'] - track2['X'].shift(-1)
track2['Y_Diff'] = track2['Y'] - track2['Y'].shift(-1)
track2['Z_Diff'] = track2['Z'] - track2['Z'].shift(-1)
track2['X_Diff']=track2['X_Diff'].pow(2)
track2['Y_Diff']=track2['Y_Diff'].pow(2)
track2['Z_Diff']=track2['Z_Diff'].pow(2)
track2['Dist']=track2['X_Diff']+track2['Y_Diff']+track2['Z_Diff']
track2['Dist']=track2['Dist'].pow(1./2)
track2['Dist_Total']=track2['Dist'].cumsum()

track2['Dist_Diff'] =track2.groupby(["Driver"])["Distance"].shift(-1) - track2['Distance']
track2["Dist_Diff_Sum_Lap"] = track2.groupby(["Driver","LapNumber"])["Dist_Diff"].cumsum()
track2["Dist_Diff_Sum"] = track2.groupby("Driver")["Dist_Diff"].cumsum()


track2["Dist_Lap"] = track2.groupby(["Driver","LapNumber"])["Dist"].cumsum()
track2["Dist_Lap_Total"] = track2.groupby(["Driver","LapNumber"])["Dist_Lap"].cumsum()


track2.drop(['X_Diff','Y_Diff','Z_Diff'], axis=1, inplace=True)

In [ ]:
track2[track2["Driver"]=="SAI"].tail()

,Year,GP,Driver,DriverNumber,Team,Position,Status,LapStartTime,LapEndTime,LapInstant,LapTime,LapNumber,RPM,Speed,nGear,Throttle,DRS,Distance,RelativeDistance,DriverAhead,DistanceToDriverAhead,X,Y,Z,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,Compound,TyreLife,FreshTyre,TrackStatus,Deleted,DeletedReason,Dist,Dist_Total,Dist_Diff,Dist_Diff_Sum_Lap,Dist_Diff_Sum,Dist_Lap,Dist_Lap_Total
47441,2021,Abu Dhabi Grand Prix,SAI,55,Ferrari,3,OnTrack,0 days 02:31:15.706000,0 days 02:32:44.585000,01:30:22.121000,00:01:28.879000,58,11311,209,5,100,1,302922.015541,0.999869,,1.816944,153,2043,-236,2,NaT,NaT,0 days 00:00:17.968000,0 days 00:00:38.391000,0 days 00:00:32.520000,0 days 02:31:33.697000,0 days 02:32:12.088000,0 days 02:32:44.608000,289,301,216,300,HARD,39,True,1,False,,97.087589,3025495.834918,9.938071,5189.271444,302931.957063,51794.136117,21048100.313212
47442,2021,Abu Dhabi Grand Prix,SAI,55,Ferrari,3,OnTrack,0 days 02:31:15.706000,0 days 02:32:44.585000,01:30:22.289000,00:01:28.879000,58,11631,212,5,100,1,302931.953611,0.999902,,1.816944,250,2047,-237,2,NaT,NaT,0 days 00:00:17.968000,0 days 00:00:38.391000,0 days 00:00:32.520000,0 days 02:31:33.697000,0 days 02:32:12.088000,0 days 02:32:44.608000,289,301,216,300,HARD,39,True,1,False,,33.151169,3025528.986087,3.134838,5192.406281,302935.091900,51827.287286,21099927.600498
47443,2021,Abu Dhabi Grand Prix,SAI,55,Ferrari,3,OnTrack,0 days 02:31:15.706000,0 days 02:32:44.585000,01:30:22.341000,00:01:28.879000,58,11772,216,5,100,1,302935.088449,0.999913,,1.816944,283,2050,-238,2,NaT,NaT,0 days 00:00:17.968000,0 days 00:00:38.391000,0 days 00:00:32.520000,0 days 02:31:33.697000,0 days 02:32:12.088000,0 days 02:32:44.608000,289,301,216,300,HARD,39,True,1,False,,123.328829,3025652.314915,11.470718,5203.876999,302946.562618,51950.616115,21151878.216613
47444,2021,Abu Dhabi Grand Prix,SAI,55,Ferrari,3,OnTrack,0 days 02:31:15.706000,0 days 02:32:44.585000,01:30:22.528000,00:01:28.879000,58,11913,220,5,100,1,302946.559167,0.999950,,1.816944,406,2059,-238,2,NaT,NaT,0 days 00:00:17.968000,0 days 00:00:38.391000,0 days 00:00:32.520000,0 days 02:31:33.697000,0 days 02:32:12.088000,0 days 02:32:44.608000,289,301,216,300,HARD,39,True,1,False,,22.226111,3025674.541026,2.045016,5205.922015,302948.607634,51972.842226,21203851.058838
47445,2021,Abu Dhabi Grand Prix,SAI,55,Ferrari,3,OnTrack,0 days 02:31:15.706000,0 days 02:32:44.585000,01:30:22.561000,00:01:28.879000,58,11241,222,5,100,1,302948.604183,0.999957,,1.816944,428,2062,-239,2,NaT,NaT,0 days 00:00:17.968000,0 days 00:00:38.391000,0 days 00:00:32.520000,0 days 02:31:33.697000,0 days 02:32:12.088000,0 days 02:32:44.608000,289,301,216,300,HARD,39,True,1,False,,1104.442846,3026778.983872,NaN,NaN,NaN,53077.285072,21256928.34391


In [ ]:
times=track2[["Year","GP","Driver","LapNumber","LapTime"]].drop_duplicates().sort_values(by=["LapTime"])
times["RK"] = times.groupby("Driver")["LapTime"].rank(method="dense", ascending=True)
times.sort_values(by=["LapTime"])

In [ ]:
#Speed Trace
px.line(track2[track2["LapTime"]=='00:01:26.615000'], x="Dist_Diff_Sum_Lap", y="Speed",
           color="Driver", hover_name="Driver",
           height=900, width=1500,
           )

In [ ]:
#Distance Trace
fig1=px.line(track2, x="Dist_Diff_Sum", y="LapInstant",
           color="Driver", hover_data=["Driver","LapNumber","Position","TyreLife"],
           height=900, width=1500,
           )

In [ ]:
fig1.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fig=px.scatter(track2, x="Dist_Diff_Sum", y="Driver", animation_frame="LapInstant",
           color="Driver", hover_name="Driver",
          #  size_max=55,
           range_x=[0,300000],
          #  range_y=[-6000,12000],
           height=900, width=1500,
           )
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 10
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
fig.show()

KeyboardInterrupt: 

In [ ]:
#Assign Rank for each entry point
track2["RK"] = track2.groupby("LapNumber")["LapInstant"].rank(method="dense", ascending=True)
track2["RK"]= track2["RK"].astype(int)
track2.tail()

,Year,GP,Driver,DriverNumber,Team,Position,Status,LapStartTime,LapEndTime,LapInstant,LapTime,LapNumber,RPM,Speed,nGear,Throttle,DRS,Distance,RelativeDistance,DriverAhead,DistanceToDriverAhead,X,Y,Z,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,Compound,TyreLife,FreshTyre,TrackStatus,Deleted,DeletedReason,Dist,Dist_Total,Dist_Diff,Dist_Diff_Sum_Lap,Dist_Diff_Sum,Dist_Lap,Dist_Lap_Total,RK,xx
47396,2021,Abu Dhabi Grand Prix,VER,33,Red Bull Racing,1,OnTrack,0 days 02:31:12.776000,0 days 02:32:39.394000,01:30:17.081000,00:01:26.618000,58,11397,216,5,100,1,302352.362714,0.999890,55,362.869444,227,2047,-237,4,NaT,NaT,0 days 00:00:17.476000,0 days 00:00:38.455000,0 days 00:00:30.687000,0 days 02:31:30.270000,0 days 02:32:08.725000,0 days 02:32:39.412000,301,305,222,304,SOFT,8,False,1,False,,28.017851,6054872.250409,2.978953,5193.104444,302355.344541,51892.925148,20379954.896681,763,302355.341667
47397,2021,Abu Dhabi Grand Prix,VER,33,Red Bull Racing,1,OnTrack,0 days 02:31:12.776000,0 days 02:32:39.394000,01:30:17.129000,00:01:26.618000,58,11565,221,5,100,1,302355.341667,0.999899,55,362.369444,255,2048,-237,4,NaT,NaT,0 days 00:00:17.476000,0 days 00:00:38.455000,0 days 00:00:30.687000,0 days 02:31:30.270000,0 days 02:32:08.725000,0 days 02:32:39.412000,301,305,222,304,SOFT,8,False,1,False,,108.231234,6054980.481643,10.765930,5203.870374,302366.110470,52001.156381,20431956.053062,764,302366.107597
47398,2021,Abu Dhabi Grand Prix,VER,33,Red Bull Racing,1,OnTrack,0 days 02:31:12.776000,0 days 02:32:39.394000,01:30:17.301000,00:01:26.618000,58,11016,223,5,100,1,302366.107597,0.999935,55,361.340417,363,2055,-238,4,NaT,NaT,0 days 00:00:17.476000,0 days 00:00:38.455000,0 days 00:00:30.687000,0 days 02:31:30.270000,0 days 02:32:08.725000,0 days 02:32:39.412000,301,305,222,304,SOFT,8,False,1,False,,42.190046,6055022.671689,4.237959,5208.108333,302370.348429,52043.346428,20483999.39949,765,302370.345556
47399,2021,Abu Dhabi Grand Prix,VER,33,Red Bull Racing,1,OnTrack,0 days 02:31:12.776000,0 days 02:32:39.394000,01:30:17.368000,00:01:26.618000,58,10467,226,6,100,1,302370.345556,0.999949,55,360.311389,405,2059,-238,4,NaT,NaT,0 days 00:00:17.476000,0 days 00:00:38.455000,0 days 00:00:30.687000,0 days 02:31:30.270000,0 days 02:32:08.725000,0 days 02:32:39.412000,301,305,222,304,SOFT,8,False,1,False,,97.744565,6055120.416254,9.777894,5217.886227,302380.126323,52141.090993,20536140.490483,766,302380.123449
47400,2021,Abu Dhabi Grand Prix,VER,33,Red Bull Racing,1,OnTrack,0 days 02:31:12.776000,0 days 02:32:39.394000,01:30:17.521000,00:01:26.618000,58,10570,228,6,100,1,302380.123449,0.999981,55,360.311389,502,2071,-239,4,NaT,NaT,0 days 00:00:17.476000,0 days 00:00:38.455000,0 days 00:00:30.687000,0 days 02:31:30.270000,0 days 02:32:08.725000,0 days 02:32:39.412000,301,305,222,304,SOFT,8,False,1,False,,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,767,NaN


In [ ]:
#Plot for a specific Lap
px.scatter(track2[track2["LapNumber"]=="2"], x="X", y="Y",
        hover_data=["GP","Driver","Team","Compound","Position","Status","LapStartTime","LapEndTime","LapInstant","LapNumber","PitOutTime","PitInTime","FreshTyre","TyreLife","Sector1Time", "Sector2Time",'Sector3Time','RK'],
        height=1500, width=1500,
        )